## Final Project Submission

* John Paul Hernandez Alcala
* Part Time
* Instructor name: Eli
* Blog posts URL:
https://john-paul-hernandez-alcala.medium.com/how-to-use-the-omdb-api-in-python-and-updating-data-mining-process-part-1-of-2-1ce48e4757be


### Libraries used

In [11]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn
import itertools as it
import requests


### Import Cleaned Budget Data

First we are gonna bring in the movie budget dataframe we obtained from MovieBudgetData.ipynb to use with the OMDb API

In [12]:
%%capture 
%run ./MovieBudgetData.ipynb

In [13]:
%store -r dfmoviebudget
dfmoviebudget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


### OMDb API Usuage

#### Get API Key to Request Data From OMDb API 

In [14]:
f = open('C:/Users/johnh/.secret/OMDb_API.txt', 'r') #requires valid API_KEY to run
API_KEY = f.read()

Now we search each of the movies through the API

In [15]:
#We would first try with a small sample first to make sure everything will go smoothly for the rest of the data
df_movie_finances = dfmoviebudget.copy() #Then progress to all of the data
print(df_movie_finances.info())
df_movie_finances.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5126 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5126 non-null   int64 
 1   release_date       5126 non-null   object
 2   movie              5126 non-null   object
 3   production_budget  5126 non-null   int64 
 4   domestic_gross     5126 non-null   int64 
 5   worldwide_gross    5126 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 280.3+ KB
None


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


We will use tuples of the moive title and year to deal with our issues of duplicate movie titles in our budget data.

In [16]:
#we will make a list of our movie titles
movieTitles = list(df_movie_finances.movie)

#Below looks at the release_date column of each movie and splits off the year for each moive title
movieYears = [df_movie_finances.release_date.iloc[x].split()[2] for x in range(0, len(movieTitles))]

I was not exactly sure how to convert two list to tuples. Here is the solution I uncovered [1] 

In [17]:
movTY = list(zip(movieTitles, movieYears)) #the zip() function helps match the title and year
movTY

[('Avatar', '2009'),
 ('Pirates of the Caribbean: On Stranger Tides', '2011'),
 ('Dark Phoenix', '2019'),
 ('Avengers: Age of Ultron', '2015'),
 ('Star Wars Ep. VIII: The Last Jedi', '2017'),
 ('Star Wars Ep. VII: The Force Awakens', '2015'),
 ('Avengers: Infinity War', '2018'),
 ('Justice League', '2017'),
 ('Spectre', '2015'),
 ('The Dark Knight Rises', '2012'),
 ('Solo: A Star Wars Story', '2018'),
 ('The Lone Ranger', '2013'),
 ('John Carter', '2012'),
 ('Tangled', '2010'),
 ('Spider-Man 3', '2007'),
 ('Captain America: Civil War', '2016'),
 ('Batman v Superman: Dawn of Justice', '2016'),
 ('The Hobbit: An Unexpected Journey', '2012'),
 ('Harry Potter and the Half-Blood Prince', '2009'),
 ('The Hobbit: The Desolation of Smaug', '2013'),
 ('The Hobbit: The Battle of the Five Armies', '2014'),
 ('The Fate of the Furious', '2017'),
 ('Superman Returns', '2006'),
 ('Pirates of the Caribbean: Dead Men Tell No Tales', '2017'),
 ('Quantum of Solace', '2008'),
 ('The Avengers', '2012'),
 (

In [18]:
movTY[0] #how to access one entity in a tuple

('Avatar', '2009')

In [19]:
len(movTY) #Makes sure we still have all our data

5126

Now we take a look at what the data looks like coming from OMDb

In [20]:
title = 'Avatar'
year = '2009'
movieInfo = requests.get('http://www.omdbapi.com/?apikey='+API_KEY+'&t='+title+'&y='+year).json()
movieInfo

{'Title': 'Avatar',
 'Year': '2009',
 'Rated': 'PG-13',
 'Released': '18 Dec 2009',
 'Runtime': '162 min',
 'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
 'Director': 'James Cameron',
 'Writer': 'James Cameron',
 'Actors': 'Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang',
 'Plot': 'A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.',
 'Language': 'English, Spanish',
 'Country': 'USA',
 'Awards': 'Won 3 Oscars. Another 86 wins & 130 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTYwOTEwNjAzMl5BMl5BanBnXkFtZTcwODc5MTUwMw@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '82%'},
  {'Source': 'Metacritic', 'Value': '83/100'}],
 'Metascore': '83',
 'imdbRating': '7.8',
 'imdbVotes': '1,129,705',
 'imdbID': 'tt0499549',
 'Type': 'movie',
 'DVD': '10 Feb 2016',
 'BoxOffic

We will use the above as a template for our future dataframe (i.e. keep the keys for use as column names)

In [21]:
movieDetails = movieInfo.copy()

Next, we will define a function that will make sure each key from our template (movie_dict_template) will match each key from the OMDb data we request (incoming_movie). Additionally, it will create fields that the template may not have already had.

In [22]:
def moviesDict(movie_dict_template, incoming_movie, OrgMovieTitle):
    '''
    movie_dict_template -- a dictionary or JSON input with keys and values already in place
    ex from OMDb response:
    {'Title': 'Avatar',
     'Year': '2009',
     'Rated': 'PG-13',
     'Released': '18 Dec 2009',
     'Runtime': '162 min',...}
    
    NOTE: must have a 'Title' key for template input or make change in code
    incoming_movie -- a dictionary or json input that will be appended to movie_dict_template
    NOTEE: incoming_movie should be similary formatted to movie_dict_template inputaq
    OrgMovieTitle -- a string of the movie title
    -----------------------------------------------------------------------------------------
    '''
    
    #checks to make sure template data has every column as a list to append incoming movie data
    if type(list(movie_dict_template.values())[0]) == str: #If first key values not list then the rest of columns' values not list
        for key, value in movie_dict_template.items():
            value = [value]
            movie_dict_template[key] = value
    else: #Every column is already a list to add other movies to
        ;
    
    #This fills in fields that might not have been present in incoming movie
    for key, value in movie_dict_template.items(): #Goes through each key in template
        if incoming_movie.get(key): #Checks if template key is present in JSON response
            #Do nothing since this key is present;
            ;
        else:#create this key so our column sizes match for each movie title
            incoming_movie[key] = None #This will also fill in movies that do not come up in the API
    
    #This fills in fields that might not have been present in template
    for key, value in incoming_movie.items(): #Goes through each key in incoming movie
        if movie_dict_template.get(key): #Checks if incoming movie key is present in template
            #Do nothing since this key is present;
            ;
        else:#create this key so our column sizes match for each movie title
            incoming_movie[key] = None
               
    #Append incoming movie to template
    for key, value in movie_dict_template.items():
        for key2, value2 in incoming_movie.items():
            if key == key2: #match up the keys from the incoming movie and template
                #This appends the incomeing movie value to the template value(s)
                if key == 'Title': #checks for title that came from incoming movie as 'NaN'
                    if value2 != None:#if title does not have 'NaN' put original title instead
                        movie_dict_template[key].append(OrgMovieTitle)
                    else:#if it does have 'NaN' leave it in there
                        movie_dict_template[key].append(value2)
                        
                else: #Addes values to keys other than the title key.
                    movie_dict_template[key].append(value2)
                    
    #returns newly appeneded template               
    return movie_dict_template

Now we see our function in action for each movie title and associated year.

In [580]:
for mov in movTY[1:]: #start at index 1 because we already have first entry
    movieInfo2 = requests.get('http://www.omdbapi.com/?apikey='+API_KEY+'&t='+mov[0]+'&y='+mov[1]).json()
    movieDetails = moviesDict(movieDetails, movieInfo2, mov[0])

#If error produced it is because API requesting requires subscription to OMDb API

Then we convert the data from movieDetails to a Pandas DataFrame

In [581]:
dfmovieDetails = pd.DataFrame.from_dict(movieDetails)
dfmovieDetails

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83,7.8,"1,129,705",tt0499549,movie,10 Feb 2016,"$760,507,625","Dune, Lightstorm Entertainment, Ingenious Film...",N/A,True
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,45,6.6,"486,087",tt1298650,movie,07 Aug 2015,"$241,071,802","Walt Disney Pictures, Jerry Bruckheimer Films",N/A,True
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,43,5.7,"159,973",tt6565702,movie,03 Sep 2019,"$65,845,974","TSG Entertainment, 20th Century Fox, Donners&#...",N/A,True
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,66,7.3,"759,059",tt2395427,movie,08 Sep 2015,"$459,005,868",Marvel Studios,N/A,True
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
5122,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
5123,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
5124,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False


In [582]:
dfmovieDetails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5126 entries, 0 to 5125
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       4392 non-null   object
 1   Year        4392 non-null   object
 2   Rated       4392 non-null   object
 3   Released    4392 non-null   object
 4   Runtime     4392 non-null   object
 5   Genre       4392 non-null   object
 6   Director    4392 non-null   object
 7   Writer      4392 non-null   object
 8   Actors      4392 non-null   object
 9   Plot        4392 non-null   object
 10  Language    4392 non-null   object
 11  Country     4392 non-null   object
 12  Awards      4392 non-null   object
 13  Poster      4392 non-null   object
 14  Ratings     4310 non-null   object
 15  Metascore   4392 non-null   object
 16  imdbRating  4392 non-null   object
 17  imdbVotes   4392 non-null   object
 18  imdbID      4392 non-null   object
 19  Type        4392 non-null   object
 20  DVD     

In [583]:
#dfmovieDetails.to_csv('OMDb API Data') #only run if the above has valid API Key and no errors.

## Clean and Export OMDb Data

As you can see on the fifth row, there is a moive which did not get recognized by the OMDb API. We need to clean these out. We noice that the issue is that titles are not being recognized. So let us look at what movies are not being found

In [23]:
dfmovieDetails = pd.read_csv('./OMDb API Data', index_col=0) #we need index_col = 0 to get rid of index column (i.e. unnamed: 0 column) [2]
dfmovieDetails

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83.0,7.8,"1,129,705",tt0499549,movie,10 Feb 2016,"$760,507,625","Dune, Lightstorm Entertainment, Ingenious Film...",NaN,True
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,45.0,6.6,"486,087",tt1298650,movie,07 Aug 2015,"$241,071,802","Walt Disney Pictures, Jerry Bruckheimer Films",NaN,True
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,43.0,5.7,"159,973",tt6565702,movie,03 Sep 2019,"$65,845,974","TSG Entertainment, 20th Century Fox, Donners&#...",NaN,True
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,66.0,7.3,"759,059",tt2395427,movie,08 Sep 2015,"$459,005,868",Marvel Studios,NaN,True
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


Let's look at all the movie indices that have the title 'NaN'

In [24]:
NAN_indices = dfmovieDetails[dfmovieDetails.Title.isna()].index
NAN_indices

Int64Index([   4,    5,   60,  129,  135,  200,  252,  254,  256,  270,
            ...
            5109, 5110, 5112, 5117, 5119, 5121, 5122, 5123, 5124, 5125],
           dtype='int64', length=734)

We drop all the rows with 'Title' as NaN.

In [25]:
dfmovieDetails.dropna(subset=['Title'], inplace=True)
dfmovieDetails.reset_index(drop=True, inplace=True)
movieDetails = pd.DataFrame.to_dict(dfmovieDetails, orient='list')
movieDetails

{'Title': ['Avatar',
  'Pirates of the Caribbean: On Stranger Tides',
  'Dark Phoenix',
  'Avengers: Age of Ultron',
  'Avengers: Infinity War',
  'Justice League',
  'Spectre',
  'The Dark Knight Rises',
  'Solo: A Star Wars Story',
  'The Lone Ranger',
  'John Carter',
  'Tangled',
  'Spider-Man 3',
  'Captain America: Civil War',
  'Batman v Superman: Dawn of Justice',
  'The Hobbit: An Unexpected Journey',
  'Harry Potter and the Half-Blood Prince',
  'The Hobbit: The Desolation of Smaug',
  'The Hobbit: The Battle of the Five Armies',
  'The Fate of the Furious',
  'Superman Returns',
  'Pirates of the Caribbean: Dead Men Tell No Tales',
  'Quantum of Solace',
  'The Avengers',
  'Man of Steel',
  'The Chronicles of Narnia: Prince Caspian',
  'The Amazing Spider-Man',
  'Battleship',
  'Transformers: The Last Knight',
  'Jurassic World',
  'Men in Black 3',
  'Transformers: Revenge of the Fallen',
  'Transformers: Age of Extinction',
  'X-Men: The Last Stand',
  'Robin Hood',
  'K

In [26]:
#Note where this dataframe stops
dfmovieDetails.tail()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
4387,Cure,2001,NaN,11 Dec 2001,68 min,Music,"Tim Pope, Richard Heslop, David G. Hillier",NaN,"Robert Smith, Perry Bamonte, Jason Cooper, The...",NaN,...,NaN,8.1,127,tt0477285,movie,12 Apr 2005,NaN,NaN,NaN,True
4388,On the Downlow,2004,Not Rated,11 Jun 2004,84 min,Drama,Tadeo Garcia,"Roger B. Domian (screenplay), Roger B. Domian ...","Tony Sancho, Michael Cortez, Donato Cruz, Feli...",Isaac and Angel are two young Latinos involved...,...,NaN,5.7,168,tt0390323,movie,11 Jan 2016,NaN,NaN,NaN,True
4389,Bang,1996,NaN,NaN,85 min,Action,King Jeff,King Jeff,"Gorio, King Jeff, Eddy Lacy, El Timo",Four police officers are kidnapped by gang mem...,...,NaN,4.2,17,tt0481267,movie,NaN,NaN,NaN,NaN,True
4390,Newlyweds,2012–,NaN,NaN,60 min,Reality-TV,NaN,NaN,"Rob Brann, Rochelle Brann, Adonis Gladney, Eri...",Explores the ups and downs of marriage as dive...,...,NaN,5.6,37,tt2010510,series,NaN,NaN,NaN,NaN,True
4391,Primer,2004,PG-13,27 May 2005,77 min,"Drama, Sci-Fi, Thriller",Shane Carruth,Shane Carruth,"Shane Carruth, David Sullivan, Casey Gooden, A...","Four friends/fledgling entrepreneurs, knowing ...",...,68.0,6.9,"99,833",tt0390384,movie,18 Nov 2016,"$424,760",NaN,NaN,True


Next we will iterate through all the movie titles to obtain a list of movies that came up as 'NaN' from OMDb

In [27]:
NAN_movieTitles = [movieTitles[i] for i in NAN_indices]
NAN_movieTitles

['Star Wars Ep. VIII: The Last Jedi',
 'Star Wars Ep. VII: The Force Awakens',
 'Prince of Persia: Sands of Time',
 'Cowboys and Aliens',
 'Fast and Furious 6',
 'Men in Black 2',
 'Harry Potter and the Deathly Hallows: Part II',
 'Harry Potter and the Deathly Hallows: Part I',
 'X-Men 2',
 'Monster Trucks',
 'Mission: Impossible 2',
 'Fantastic Four: Rise of the Silver Surfer',
 'Star Wars Ep. I: The Phantom Menace',
 'Star Wars Ep. III: Revenge of the Sith',
 'Star Wars Ep. II: Attack of the Clones',
 'Mr. and Mrs. Smith',
 'The Divergent Series: Allegiant',
 'The Hangover 3',
 "Lemony Snicket's A Series of Unfortunate Events",
 'Seventh Son',
 'Spider-Man: Into The Spider-Verse 3D',
 'Artificial Intelligence: AI',
 'The Nutcracker in 3D',
 'I Now Pronounce You Chuck and Larry',
 'Fantasia 2000 (Theatrical Release)',
 'Battlefield Earth: A Saga of the Year 3000',
 'Arthur et les Minimoys',
 'The Little Prince',
 'AVP: Alien Vs. Predator',
 'Star Trek: Insurrection',
 'Wall Street 2: 

In [28]:
NAN_movieYears = [movieYears[i] for i in NAN_indices]
NAN_movieYears

['2017',
 '2015',
 '2010',
 '2011',
 '2013',
 '2002',
 '2011',
 '2010',
 '2003',
 '2017',
 '2000',
 '2007',
 '1999',
 '2005',
 '2002',
 '2005',
 '2016',
 '2013',
 '2004',
 '2015',
 '2018',
 '2001',
 '2010',
 '2007',
 '1999',
 '2000',
 '2007',
 '2016',
 '2004',
 '1998',
 '2010',
 '2012',
 '2015',
 '2009',
 '2005',
 '2009',
 '2017',
 '2008',
 '2016',
 '2008',
 '2016',
 '2015',
 '2003',
 '2011',
 '1981',
 '2002',
 '2006',
 '2013',
 '2004',
 '2014',
 '2016',
 '2005',
 '2009',
 '2009',
 '2015',
 '2001',
 '2007',
 '2013',
 '1982',
 '2010',
 '2005',
 '2005',
 '2005',
 '2006',
 '2005',
 '2016',
 '2017',
 '2013',
 '2011',
 '1999',
 '2006',
 '2007',
 '2013',
 '2002',
 '1997',
 '2000',
 '2019',
 '2011',
 '2016',
 '2009',
 '2005',
 '2017',
 '2010',
 '2000',
 '1993',
 '1996',
 '2009',
 '2014',
 '2006',
 '2015',
 '1983',
 '2005',
 '2005',
 '2017',
 '1999',
 '2004',
 '2010',
 '2011',
 '2017',
 '2011',
 '2006',
 '2018',
 '2007',
 '2010',
 '2011',
 '2008',
 '2015',
 '1998',
 '2003',
 '2002',
 '2011',
 

In [29]:
#We just use titles this time
for MOV in NAN_movieTitles:
    NAN_movieInfo2 = requests.get('http://www.omdbapi.com/?apikey='+API_KEY+'&t='+MOV).json()
    movieDetails = moviesDict(movieDetails, NAN_movieInfo2, MOV)

#If error produced it is because API requesting requires subscription to OMDb API

In [30]:
df_NAN_movieDetails = pd.DataFrame.from_dict(movieDetails)
df_NAN_movieDetails

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83,7.8,"1,129,705",tt0499549,movie,10 Feb 2016,"$760,507,625","Dune, Lightstorm Entertainment, Ingenious Film...",NaN,True
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,45,6.6,"486,087",tt1298650,movie,07 Aug 2015,"$241,071,802","Walt Disney Pictures, Jerry Bruckheimer Films",NaN,True
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,43,5.7,"159,973",tt6565702,movie,03 Sep 2019,"$65,845,974","TSG Entertainment, 20th Century Fox, Donners&#...",NaN,True
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,66,7.3,"759,059",tt2395427,movie,08 Sep 2015,"$459,005,868",Marvel Studios,NaN,True
4,Avengers: Infinity War,2018,PG-13,27 Apr 2018,149 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",The Avengers and their allies must be willing ...,...,68,8.4,"865,422",tt4154756,movie,31 Jul 2018,"$678,815,482",Marvel Studios,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,Cavite,2005,Not Rated,12 Mar 2005,80 min,Thriller,"Neill Dela Llana, Ian Gamazon","Neill Dela Llana, Ian Gamazon","Ian Gamazon, Edwin Lagda, Edgar Tancangco, Nor...","Flying back to his home country, an American m...",...,64,6.3,604,tt0428303,movie,02 Dec 2016,N/A,N/A,N/A,True
5122,The Mongol King,2005,PG-13,23 Apr 2005,84 min,"Drama, Crime",Anthony Vallone,"Richard Jewell, Robert Morgan, Anthony Vallone","John Considine, Gerry Cooper, Carey Crim, Jess...",While writing a term paper a woman gets involv...,...,N/A,7.8,36,tt0430371,movie,N/A,N/A,N/A,N/A,True
5123,Following,1998,R,05 Nov 1999,69 min,"Crime, Mystery, Thriller",Christopher Nolan,Christopher Nolan,"Jeremy Theobald, Alex Haw, Lucy Russell, John ...",A young writer who follows strangers for mater...,...,60,7.5,"88,179",tt0154506,movie,10 Mar 2017,"$48,482",Next Wave Films,N/A,True
5124,Return to the Land of Wonders,2004,N/A,27 Apr 2004,88 min,"Documentary, War",Maysoon Pachachi,N/A,N/A,Moving between the political sphere and everyd...,...,60,6.9,37,tt0461832,movie,13 Dec 2005,"$1,338",Arab Film Distribution,N/A,True


In [31]:
#Where our last df ended
df_NAN_movieDetails[df_NAN_movieDetails.Title == 'Primer']

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
4391,Primer,2004,PG-13,27 May 2005,77 min,"Drama, Sci-Fi, Thriller",Shane Carruth,Shane Carruth,"Shane Carruth, David Sullivan, Casey Gooden, A...","Four friends/fledgling entrepreneurs, knowing ...",...,68,6.9,"99,833",tt0390384,movie,18 Nov 2016,"$424,760",NaN,NaN,True


In [32]:
df_NAN_movieDetails.iloc[4391,:]

Title                                                    Primer
Year                                                       2004
Rated                                                     PG-13
Released                                            27 May 2005
Runtime                                                  77 min
Genre                                   Drama, Sci-Fi, Thriller
Director                                          Shane Carruth
Writer                                            Shane Carruth
Actors        Shane Carruth, David Sullivan, Casey Gooden, A...
Plot          Four friends/fledgling entrepreneurs, knowing ...
Language                                        English, French
Country                                                     USA
Awards                                  3 wins & 7 nominations.
Poster        https://m.media-amazon.com/images/M/MV5BNjc3OW...
Ratings       [{'Source': 'Internet Movie Database', 'Value'...
Metascore                               

In [33]:
NAN_indices = df_NAN_movieDetails[df_NAN_movieDetails.Title.isna()].index
NAN_indices

Int64Index([4392, 4393, 4394, 4395, 4396, 4397, 4398, 4399, 4400, 4402,
            ...
            5049, 5051, 5062, 5072, 5080, 5082, 5086, 5090, 5100, 5104],
           dtype='int64', length=221)

In [34]:
NAN_indices-4392

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,  10,
            ...
            657, 659, 670, 680, 688, 690, 694, 698, 708, 712],
           dtype='int64', length=221)

In [35]:
NAN_indices = df_NAN_movieDetails[df_NAN_movieDetails.Title.isna()].index-4392
NAN_indices

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,  10,
            ...
            657, 659, 670, 680, 688, 690, 694, 698, 708, 712],
           dtype='int64', length=221)

In [36]:
df_NAN_movieDetails.dropna(subset=['Title'], inplace=True)
df_NAN_movieDetails.reset_index(drop=True, inplace=True)
movieDetails = pd.DataFrame.to_dict(df_NAN_movieDetails, orient='list')
movieDetails

{'Title': ['Avatar',
  'Pirates of the Caribbean: On Stranger Tides',
  'Dark Phoenix',
  'Avengers: Age of Ultron',
  'Avengers: Infinity War',
  'Justice League',
  'Spectre',
  'The Dark Knight Rises',
  'Solo: A Star Wars Story',
  'The Lone Ranger',
  'John Carter',
  'Tangled',
  'Spider-Man 3',
  'Captain America: Civil War',
  'Batman v Superman: Dawn of Justice',
  'The Hobbit: An Unexpected Journey',
  'Harry Potter and the Half-Blood Prince',
  'The Hobbit: The Desolation of Smaug',
  'The Hobbit: The Battle of the Five Armies',
  'The Fate of the Furious',
  'Superman Returns',
  'Pirates of the Caribbean: Dead Men Tell No Tales',
  'Quantum of Solace',
  'The Avengers',
  'Man of Steel',
  'The Chronicles of Narnia: Prince Caspian',
  'The Amazing Spider-Man',
  'Battleship',
  'Transformers: The Last Knight',
  'Jurassic World',
  'Men in Black 3',
  'Transformers: Revenge of the Fallen',
  'Transformers: Age of Extinction',
  'X-Men: The Last Stand',
  'Robin Hood',
  'K

In [37]:
NAN_movieTitles = [NAN_movieTitles[i] for i in NAN_indices]
NAN_movieTitles

['Star Wars Ep. VIII: The Last Jedi',
 'Star Wars Ep. VII: The Force Awakens',
 'Prince of Persia: Sands of Time',
 'Cowboys and Aliens',
 'Fast and Furious 6',
 'Men in Black 2',
 'Harry Potter and the Deathly Hallows: Part II',
 'Harry Potter and the Deathly Hallows: Part I',
 'X-Men 2',
 'Mission: Impossible 2',
 'Fantastic Four: Rise of the Silver Surfer',
 'Star Wars Ep. I: The Phantom Menace',
 'Star Wars Ep. III: Revenge of the Sith',
 'Star Wars Ep. II: Attack of the Clones',
 'The Divergent Series: Allegiant',
 'The Hangover 3',
 "Lemony Snicket's A Series of Unfortunate Events",
 'Spider-Man: Into The Spider-Verse 3D',
 'Artificial Intelligence: AI',
 'The Nutcracker in 3D',
 'I Now Pronounce You Chuck and Larry',
 'Fantasia 2000 (Theatrical Release)',
 'Battlefield Earth: A Saga of the Year 3000',
 'Arthur et les Minimoys',
 'AVP: Alien Vs. Predator',
 'Wall Street 2: Money Never Sleeps',
 "Doctor Seuss' The Lorax",
 'Tian jiang xiong shi',
 'Miss Congeniality 2: Armed and F

Here we do some customized cleaning 

In [656]:
variable = "Star Wars Ep. VIII: The Last Jedi"
variable = variable.replace('Ep.','Episode')
variable

'Star Wars Episode VIII: The Last Jedi'

In [657]:
variable = 'Cowboys and Aliens'
variable = variable.replace('and', '&')
variable

'Cowboys & Aliens'

In [658]:
re.sub('and', 'and', '&')

'&'

In [659]:
variable = "Star Wars Ep. VIII: The Last Jedi"
variable.replace('and', '&')

'Star Wars Ep. VIII: The Last Jedi'

In [660]:
movie_string = 'Star Wars Ep. VIII: The Last Jedi'
if movie_string.replace(' Ep. ',' Episode ') != movie_string:
    movie_string = movie_string.replace(' Ep. ',' Episode ')
elif movie_string.replace(' and ', ' & ') != movie_string:
    movie_string = movie_string.replace(' and ', ' & ')
movie_string

'Star Wars Episode VIII: The Last Jedi'

In [39]:
def CleanString(movie_string):
    '''
    Coverts strings with numberic characters, 'Ep.', and 'and'
    to roman numerals, 'Episode', and '&', respectively
    '''
    if movie_string.replace(' Ep. ',' Episode ') != movie_string:
        movie_string = movie_string.replace('Ep.','Episode')
    elif movie_string.replace(' and ', ' & ') != movie_string:
        movie_string = movie_string.replace(' and ', ' & ')
    return movie_string

In [40]:
reformatted_NAN_movies = [CleanString(x) for x in NAN_movieTitles]
reformatted_NAN_movies 

['Star Wars Episode VIII: The Last Jedi',
 'Star Wars Episode VII: The Force Awakens',
 'Prince of Persia: Sands of Time',
 'Cowboys & Aliens',
 'Fast & Furious 6',
 'Men in Black 2',
 'Harry Potter & the Deathly Hallows: Part II',
 'Harry Potter & the Deathly Hallows: Part I',
 'X-Men 2',
 'Mission: Impossible 2',
 'Fantastic Four: Rise of the Silver Surfer',
 'Star Wars Episode I: The Phantom Menace',
 'Star Wars Episode III: Revenge of the Sith',
 'Star Wars Episode II: Attack of the Clones',
 'The Divergent Series: Allegiant',
 'The Hangover 3',
 "Lemony Snicket's A Series of Unfortunate Events",
 'Spider-Man: Into The Spider-Verse 3D',
 'Artificial Intelligence: AI',
 'The Nutcracker in 3D',
 'I Now Pronounce You Chuck & Larry',
 'Fantasia 2000 (Theatrical Release)',
 'Battlefield Earth: A Saga of the Year 3000',
 'Arthur et les Minimoys',
 'AVP: Alien Vs. Predator',
 'Wall Street 2: Money Never Sleeps',
 "Doctor Seuss' The Lorax",
 'Tian jiang xiong shi',
 'Miss Congeniality 2: A

In [42]:
#This will ensure we keep the original titles for when we combine the dataframes
movieTitles_orig_and_reformatted = list(zip(NAN_movieTitles, reformatted_NAN_movies ))
movieTitles_orig_and_reformatted

[('Star Wars Ep. VIII: The Last Jedi',
  'Star Wars Episode VIII: The Last Jedi'),
 ('Star Wars Ep. VII: The Force Awakens',
  'Star Wars Episode VII: The Force Awakens'),
 ('Prince of Persia: Sands of Time', 'Prince of Persia: Sands of Time'),
 ('Cowboys and Aliens', 'Cowboys & Aliens'),
 ('Fast and Furious 6', 'Fast & Furious 6'),
 ('Men in Black 2', 'Men in Black 2'),
 ('Harry Potter and the Deathly Hallows: Part II',
  'Harry Potter & the Deathly Hallows: Part II'),
 ('Harry Potter and the Deathly Hallows: Part I',
  'Harry Potter & the Deathly Hallows: Part I'),
 ('X-Men 2', 'X-Men 2'),
 ('Mission: Impossible 2', 'Mission: Impossible 2'),
 ('Fantastic Four: Rise of the Silver Surfer',
  'Fantastic Four: Rise of the Silver Surfer'),
 ('Star Wars Ep. I: The Phantom Menace',
  'Star Wars Episode I: The Phantom Menace'),
 ('Star Wars Ep. III: Revenge of the Sith',
  'Star Wars Episode III: Revenge of the Sith'),
 ('Star Wars Ep. II: Attack of the Clones',
  'Star Wars Episode II: Atta

In [43]:
movieTitles_orig_and_reformatted[0][0], movieTitles_orig_and_reformatted[0][1]

('Star Wars Ep. VIII: The Last Jedi', 'Star Wars Episode VIII: The Last Jedi')

In [663]:
for mov in movieTitles_orig_and_reformatted:
    NAN_movieInfo2 = requests.get('http://www.omdbapi.com/?apikey='+API_KEY+'&t='+mov[1]).json()
    movieDetails = moviesDict(movieDetails, NAN_movieInfo2, move[0])

#If error produced it is because API requesting requires subscription to OMDb API

In [664]:
df_NAN_movieDetails = pd.DataFrame.from_dict(movieDetails)
df_NAN_movieDetails

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83,7.8,"1,129,705",tt0499549,movie,10 Feb 2016,"$760,507,625","Dune, Lightstorm Entertainment, Ingenious Film...",NaN,True
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,45,6.6,"486,087",tt1298650,movie,07 Aug 2015,"$241,071,802","Walt Disney Pictures, Jerry Bruckheimer Films",NaN,True
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,43,5.7,"159,973",tt6565702,movie,03 Sep 2019,"$65,845,974","TSG Entertainment, 20th Century Fox, Donners&#...",NaN,True
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,66,7.3,"759,059",tt2395427,movie,08 Sep 2015,"$459,005,868",Marvel Studios,NaN,True
4,Avengers: Infinity War,2018,PG-13,27 Apr 2018,149 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",The Avengers and their allies must be willing ...,...,68,8.4,"865,422",tt4154756,movie,31 Jul 2018,"$678,815,482",Marvel Studios,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
5122,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
5123,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
5124,"Peace, Propaganda & the Promised Land",2004,N/A,N/A,80 min,Documentary,"Sut Jhally, Bathsheba Ratzkoff","Sut Jhally, Bathsheba Ratzkoff","Seth Ackerman, Stav Adivi, Arik Ascherman, Han...",This video shows how the foreign policy intere...,...,57,8.2,531,tt0428959,movie,N/A,"$4,930",N/A,N/A,True


In [665]:
NAN_indices = df_NAN_movieDetails[df_NAN_movieDetails.Title.isna()].index
NAN_indices

Int64Index([4907, 4910, 4913, 4914, 4915, 4919, 4920, 4921, 4922, 4923,
            ...
            5113, 5114, 5115, 5116, 5117, 5118, 5120, 5121, 5122, 5123],
           dtype='int64', length=183)

In [668]:
df_NAN_movieDetails.dropna(subset=['Title'], inplace=True)
df_NAN_movieDetails.reset_index(drop=True, inplace=True)

df_NAN_movieDetails

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83,7.8,"1,129,705",tt0499549,movie,10 Feb 2016,"$760,507,625","Dune, Lightstorm Entertainment, Ingenious Film...",NaN,True
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,45,6.6,"486,087",tt1298650,movie,07 Aug 2015,"$241,071,802","Walt Disney Pictures, Jerry Bruckheimer Films",NaN,True
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,43,5.7,"159,973",tt6565702,movie,03 Sep 2019,"$65,845,974","TSG Entertainment, 20th Century Fox, Donners&#...",NaN,True
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,66,7.3,"759,059",tt2395427,movie,08 Sep 2015,"$459,005,868",Marvel Studios,NaN,True
4,Avengers: Infinity War,2018,PG-13,27 Apr 2018,149 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",The Avengers and their allies must be willing ...,...,68,8.4,"865,422",tt4154756,movie,31 Jul 2018,"$678,815,482",Marvel Studios,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4938,Travellers & Magicians,1992,N/A,01 Feb 1994,90 min,Drama,Bahram Beizai,Bahram Beizai,"Mozhdeh Shamsai, Homa Rusta, Jamileh Sheikhi, ...",A young woman's wedding becomes a ritual of mo...,...,N/A,7.4,906,tt0104919,movie,N/A,N/A,N/A,N/A,True
4939,Celeste & Jesse Forever,2018,TV-14,06 Feb 2019,105 min,"Drama, Music",Ben Hackworth,"Bille Brown, Ben Hackworth","Radha Mitchell, Thomas Cocquerel, Nadine Garne...",Celeste is a love story set in the tropical sp...,...,N/A,5.5,153,tt5705306,movie,29 May 2019,N/A,Unicorn Films,N/A,True
4940,Lovely & Amazing,2012,N/A,30 Mar 2012,139 min,Romance,Jaya B.,"Jaya B. (story & screenplay), Shyam Manohar (d...","Aadi, Shanvi Srivastava, Rajendra Prasad, Chin...",N/A,...,N/A,5.4,155,tt2334727,movie,N/A,N/A,N/A,N/A,True
4941,"Peace, Propaganda & the Promised Land",2004,N/A,N/A,80 min,Documentary,"Sut Jhally, Bathsheba Ratzkoff","Sut Jhally, Bathsheba Ratzkoff","Seth Ackerman, Stav Adivi, Arik Ascherman, Han...",This video shows how the foreign policy intere...,...,57,8.2,531,tt0428959,movie,N/A,"$4,930",N/A,N/A,True


We know that since there was index error that all the 'NaN' movies have been identified.

In [669]:
df_NAN_movieDetails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4943 entries, 0 to 4942
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       4943 non-null   object
 1   Year        4943 non-null   object
 2   Rated       4729 non-null   object
 3   Released    4886 non-null   object
 4   Runtime     4898 non-null   object
 5   Genre       4935 non-null   object
 6   Director    4874 non-null   object
 7   Writer      4806 non-null   object
 8   Actors      4915 non-null   object
 9   Plot        4845 non-null   object
 10  Language    4928 non-null   object
 11  Country     4940 non-null   object
 12  Awards      4334 non-null   object
 13  Poster      4832 non-null   object
 14  Ratings     4849 non-null   object
 15  Metascore   4414 non-null   object
 16  imdbRating  4859 non-null   object
 17  imdbVotes   4859 non-null   object
 18  imdbID      4943 non-null   object
 19  Type        4943 non-null   object
 20  DVD     

From above, we can see that we have less than 4% of our data as NaN! Depending on our analysis, we will drop certain columns. For our analysis, we can drop the 'website' column because it has very little valid points and the 'response' column does not relate to the actual movie information.

In [670]:
df_NAN_movieDetails.drop(['Website', 'Response'], axis=1, inplace=True)#inplace being true keeps change

In [682]:
df_NAN_movieDetails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4943 entries, 0 to 4942
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       4943 non-null   object
 1   Year        4943 non-null   object
 2   Rated       4729 non-null   object
 3   Released    4886 non-null   object
 4   Runtime     4898 non-null   object
 5   Genre       4935 non-null   object
 6   Director    4874 non-null   object
 7   Writer      4806 non-null   object
 8   Actors      4915 non-null   object
 9   Plot        4845 non-null   object
 10  Language    4928 non-null   object
 11  Country     4940 non-null   object
 12  Awards      4334 non-null   object
 13  Poster      4832 non-null   object
 14  Ratings     4849 non-null   object
 15  Metascore   4414 non-null   object
 16  imdbRating  4859 non-null   object
 17  imdbVotes   4859 non-null   object
 18  imdbID      4943 non-null   object
 19  Type        4943 non-null   object
 20  DVD     

In [675]:
df10 = df_NAN_movieDetails.copy()
df10.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,https://m.media-amazon.com/images/M/MV5BMTYwOT...,"[{'Source': 'Internet Movie Database', 'Value'...",83,7.8,"1,129,705",tt0499549,movie,10 Feb 2016,"$760,507,625","Dune, Lightstorm Entertainment, Ingenious Film..."
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,https://m.media-amazon.com/images/M/MV5BMjE5Mj...,"[{'Source': 'Internet Movie Database', 'Value'...",45,6.6,"486,087",tt1298650,movie,07 Aug 2015,"$241,071,802","Walt Disney Pictures, Jerry Bruckheimer Films"
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,https://m.media-amazon.com/images/M/MV5BMmZmYT...,"[{'Source': 'Internet Movie Database', 'Value'...",43,5.7,"159,973",tt6565702,movie,03 Sep 2019,"$65,845,974","TSG Entertainment, 20th Century Fox, Donners&#..."
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,https://m.media-amazon.com/images/M/MV5BMTM4OG...,"[{'Source': 'Internet Movie Database', 'Value'...",66,7.3,"759,059",tt2395427,movie,08 Sep 2015,"$459,005,868",Marvel Studios
4,Avengers: Infinity War,2018,PG-13,27 Apr 2018,149 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",The Avengers and their allies must be willing ...,...,https://m.media-amazon.com/images/M/MV5BMjMxNj...,"[{'Source': 'Internet Movie Database', 'Value'...",68,8.4,"865,422",tt4154756,movie,31 Jul 2018,"$678,815,482",Marvel Studios


In [683]:
df_NAN_movieDetails.to_csv('ImprovedReadyOMDbAPIData')

In [OMDbBudgetDataAnalysis.ipynb](OMDbBudgetDataAnalysis.ipynb), we will use 'ImprovedReadyOMDbAPIData' along with R.O.I. data to conduct a further analysis.

## Resources used for development:
1. https://docs.quantifiedcode.com/python-anti-patterns/readability/not_using_zip_to_iterate_over_a_pair_of_lists.html
2. https://stackoverflow.com/questions/36519086/how-to-get-rid-of-unnamed-0-column-in-a-pandas-dataframe
3. http://www.omdbapi.com/